In [1]:
from infoseq.bpe import bpe_from_text

from infoseq.dataset import kjv_path

with open(kjv_path, 'rb') as f:
    kjv = f.read()

len(kjv)

4457889

In [5]:
bpe = bpe_from_text(kjv, max_tokens=500)
len(bpe)

500

In [7]:
[k for k in bpe.encode_map if len(k) > 1]

[b'th',
 b'e ',
 b'\r\n',
 b' t',
 b' th',
 b' a',
 b'nd',
 b'er',
 b't ',
 b'in',
 b'of',
 b's ',
 b' the',
 b', ',
 b'ou',
 b'nd ',
 b'ha',
 b' the ',
 b'd ',
 b'or',
 b'y ',
 b', a',
 b'en',
 b'll',
 b'hi',
 b'on',
 b'.\r',
 b'.\r\n',
 b'o ',
 b'the',
 b'the ',
 b'es',
 b're',
 b'he',
 b' s',
 b' w',
 b'.\r\n\r',
 b'.\r\n\r\n',
 b' A',
 b'to',
 b'an',
 b', an',
 b'ing',
 b'ar',
 b'of ',
 b', and',
 b'at',
 b' b',
 b'ed',
 b' f',
 b' m',
 b'it',
 b':1',
 b'om',
 b'un',
 b' o',
 b' of',
 b'el',
 b'he ',
 b'se',
 b'no',
 b' An',
 b' And',
 b'to ',
 b'sa',
 b'hal',
 b'hall',
 b', and ',
 b'ri',
 b'st',
 b' of ',
 b' an',
 b' and',
 b'is',
 b'and',
 b'ith',
 b' c',
 b'le',
 b'am',
 b':2',
 b' I',
 b' h',
 b'fo',
 b' d',
 b'gh',
 b' i',
 b' in',
 b'LO',
 b'ch',
 b've',
 b'im',
 b'ra',
 b'id',
 b' p',
 b'il',
 b'ea',
 b'RD',
 b'ir',
 b'ro',
 b' y',
 b'unt',
 b'up',
 b'tha',
 b'unto',
 b'ver',
 b'ut',
 b' T',
 b'li',
 b' to',
 b'ol',
 b' he',
 b'LOR',
 b'ill',
 b'LORD',
 b'al',
 b'as',
 b'f